# __THIS NOTEBOOK IS TO BE USED FOR IBM DATA SCIENCE CAPSTONE PROJECT__


In [1]:
#Import all necessary libraries/packages
import itertools
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import NullFormatter
import pandas as pd
import numpy as np
import matplotlib.ticker as ticker
from sklearn import preprocessing
%matplotlib inline
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn import svm
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import jaccard_similarity_score
from sklearn.metrics import f1_score
from sklearn.metrics import log_loss
import folium # map rendering library
import urllib.request
from geopy.geocoders import Nominatim
from pandas.io.json import json_normalize
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors
import requests # library to handle requests
print('Libraries imported.')

Libraries imported.


In [2]:
print("Hello Capstone Project Course!")

Hello Capstone Project Course!


## 1. Retrieve the Foursquare data and put it into a pandas dataframe:

In [131]:
CLIENT_ID = 'EZJXKXBQ1CNUG1GQNSNCNTOT1SWVM5BQNFNYD3D10SR4SR3L' # your Foursquare ID
CLIENT_SECRET = 'AEHEQWN50BFII1X1H3QMO55RIUFJCNL1U3RHERGKNDLZZ3D4' # your Foursquare Secret
VERSION = '20180604'
radius = 10000
LIMIT = 100

In [132]:
address = '400 Welton St, Denver, CO'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

39.7546069 -104.9783125


In [133]:
search_query = 'restaurant'

In [134]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=EZJXKXBQ1CNUG1GQNSNCNTOT1SWVM5BQNFNYD3D10SR4SR3L&client_secret=AEHEQWN50BFII1X1H3QMO55RIUFJCNL1U3RHERGKNDLZZ3D4&ll=39.7546069,-104.9783125&v=20180604&query=restaurant&radius=10000&limit=100'

In [135]:
results = requests.get(url).json()

In [136]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

C:\Users\Jake - Alienware\anaconda3\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  """


,id,name,categories,referralId,hasPerk,location.address,location.lat,location.lng,location.labeledLatLngs,location.distance,...,location.formattedAddress,delivery.id,delivery.url,delivery.provider.name,delivery.provider.icon.prefix,delivery.provider.icon.sizes,delivery.provider.icon.name,location.crossStreet,location.neighborhood,venuePage.id
0,4b75a7bff964a5208c1a2ee3,La Fiesta Mexican Restaurant,"[{'id': '4bf58dd8d48988d1c1941735', 'name': 'M...",v-1592868081,False,2340 Champa St,39.753410,-104.984636,"[{'label': 'display', 'lat': 39.75341022800047...",557,...,"[2340 Champa St, Denver, CO 80205, United States]",555642,https://www.grubhub.com/restaurant/la-fiesta-2...,grubhub,https://fastly.4sqi.net/img/general/cap/,"[40, 50]",/delivery_provider_grubhub_20180129.png,NaN,NaN,NaN
1,3fd66200f964a520b4ec1ee3,Rio Grande Mexican Restaurant,"[{'id': '4bf58dd8d48988d1c1941735', 'name': 'M...",v-1592868081,False,1525 Blake St,39.750223,-105.000036,"[{'label': 'display', 'lat': 39.75022256969623...",1922,...,"[1525 Blake St (btwn 17th St & 15th St), Denve...",306685,https://www.grubhub.com/restaurant/rio-grande-...,grubhub,https://fastly.4sqi.net/img/general/cap/,"[40, 50]",/delivery_provider_grubhub_20180129.png,btwn 17th St & 15th St,LoDo,NaN
2,4c06fc11b4aa0f475e056562,Randolph's Restaurant & Bar,"[{'id': '4bf58dd8d48988d14e941735', 'name': 'A...",v-1592868081,False,1776 Grant St,39.744531,-104.983480,"[{'label': 'display', 'lat': 39.74453056889666...",1205,...,"[1776 Grant St (at 17th), Denver, CO 80203, Un...",NaN,NaN,NaN,NaN,NaN,NaN,at 17th,NaN,36169341
3,4a77ac3ff964a520fbe41fe3,The Corner Office Restaurant & Martini Bar,"[{'id': '4bf58dd8d48988d14e941735', 'name': 'A...",v-1592868081,False,1401 Curtis St,39.745555,-104.997231,"[{'label': 'display', 'lat': 39.74555533418328...",1907,...,"[1401 Curtis St (at 14th St), Denver, CO 80202...",NaN,NaN,NaN,NaN,NaN,NaN,at 14th St,NaN,35396549
4,4cc60fb18062468804e23f2f,EDGE Restaurant And Bar At Four Seasons Hotel ...,"[{'id': '4bf58dd8d48988d1cc941735', 'name': 'S...",v-1592868081,False,1111 14th St,39.746367,-104.997919,"[{'label': 'display', 'lat': 39.74636682978978...",1912,...,"[1111 14th St (Arapahoe), Denver, CO 80202, Un...",NaN,NaN,NaN,NaN,NaN,NaN,Arapahoe,NaN,NaN


## 2. Data cleaning and filtering

### 2a. Filter only the relevant columns:

In [137]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered.head()

,name,categories,address,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,crossStreet,neighborhood,id
0,La Fiesta Mexican Restaurant,Mexican Restaurant,2340 Champa St,39.753410,-104.984636,"[{'label': 'display', 'lat': 39.75341022800047...",557,80205,US,Denver,CO,United States,"[2340 Champa St, Denver, CO 80205, United States]",NaN,NaN,4b75a7bff964a5208c1a2ee3
1,Rio Grande Mexican Restaurant,Mexican Restaurant,1525 Blake St,39.750223,-105.000036,"[{'label': 'display', 'lat': 39.75022256969623...",1922,80202,US,Denver,CO,United States,"[1525 Blake St (btwn 17th St & 15th St), Denve...",btwn 17th St & 15th St,LoDo,3fd66200f964a520b4ec1ee3
2,Randolph's Restaurant & Bar,American Restaurant,1776 Grant St,39.744531,-104.983480,"[{'label': 'display', 'lat': 39.74453056889666...",1205,80203,US,Denver,CO,United States,"[1776 Grant St (at 17th), Denver, CO 80203, Un...",at 17th,NaN,4c06fc11b4aa0f475e056562
3,The Corner Office Restaurant & Martini Bar,American Restaurant,1401 Curtis St,39.745555,-104.997231,"[{'label': 'display', 'lat': 39.74555533418328...",1907,80202,US,Denver,CO,United States,"[1401 Curtis St (at 14th St), Denver, CO 80202...",at 14th St,NaN,4a77ac3ff964a520fbe41fe3
4,EDGE Restaurant And Bar At Four Seasons Hotel ...,Steakhouse,1111 14th St,39.746367,-104.997919,"[{'label': 'display', 'lat': 39.74636682978978...",1912,80202,US,Denver,CO,United States,"[1111 14th St (Arapahoe), Denver, CO 80202, Un...",Arapahoe,NaN,4cc60fb18062468804e23f2f


In [138]:
dataframe_filtered['categories'].unique()

array(['Mexican Restaurant', 'American Restaurant', 'Steakhouse',
       'Food Service', 'Indian Restaurant', 'Lounge', 'Restaurant',
       'Seafood Restaurant', 'Japanese Restaurant', 'Brewery',
       'Pizza Place', 'Food', 'Kitchen Supply Store',
       'Ethiopian Restaurant', 'African Restaurant', 'Chinese Restaurant',
       'Gastropub', 'Waste Facility', 'Korean Restaurant',
       'Furniture / Home Store', 'Office', 'Financial or Legal Service'],
      dtype=object)

### 2b. Get ratings for all venues in dataframe: 

In [139]:
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)
url

'https://api.foursquare.com/v2/venues/4c0ba8406071a5933484e132?client_id=EZJXKXBQ1CNUG1GQNSNCNTOT1SWVM5BQNFNYD3D10SR4SR3L&client_secret=AEHEQWN50BFII1X1H3QMO55RIUFJCNL1U3RHERGKNDLZZ3D4&v=20180604'

In [140]:
result = requests.get(url).json()
#print(result['response']['venue'].keys())
#result['response']['venue']

### 2c. Find the associated ratings for each venue category:

In [142]:
for i, j, e in zip(dataframe_filtered['id'], dataframe_filtered['categories'], dataframe_filtered['name'] ):
    venue_id = i
    url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)
    result = requests.get(url).json()
    try:
        print("\'",e,"\'", "--", "Category: ", j,"Rating:  ", result['response']['venue']['rating'])
    except:
        print("\'",e,"\'", "--", "Category: ", j,"Rating:  ", 'No Ratings')



' La Fiesta Mexican Restaurant ' -- Category:  Mexican Restaurant Rating:   No Ratings
' Rio Grande Mexican Restaurant ' -- Category:  Mexican Restaurant Rating:   7.6
' Randolph's Restaurant & Bar ' -- Category:  American Restaurant Rating:   6.8
' The Corner Office Restaurant & Martini Bar ' -- Category:  American Restaurant Rating:   7.6
' EDGE Restaurant And Bar At Four Seasons Hotel Denver ' -- Category:  Steakhouse Rating:   7.4
' The Restaurant Source, a division of Bargreen Ellingson ' -- Category:  Food Service Rating:   No Ratings
' Little India Restaurant - Champa Downtown ' -- Category:  Indian Restaurant Rating:   6.7
' Pete's Satire Restaurant & Lounge ' -- Category:  Lounge Rating:   5.9
' Altitude Restaurant ' -- Category:  Restaurant Rating:   6.1
' Denver Downtown Aquarium Restaurant ' -- Category:  Seafood Restaurant Rating:   7.3
' Mexico City Restaurant ' -- Category:  Mexican Restaurant Rating:   No Ratings
' Domo Japanese Country Foods Restaurant ' -- Category:  

In [143]:
mr_avg = (7.6 + 7.6 + 6.5 + 7.5 + 7.1 + 8.7 + 6.7)/8
ar_avg = (6.8+7.6+4.7)/3
s_avg = 7.4
r_avg = 6.1
ir_avg = (6.7+7.5)/3
l_avg = 5.9
jr_avg = 8.6
sr_avg = 7.3
pp_avg = 7.1
b_avg = 7.4
er_avg = (6.6+7.5)/2
bs_avg = 8.0

### 2d. Filter out only restaurants that have at least 1 rating, and load into dataframe: 

In [144]:
data = {'Venue Category':  ['Mexican Restaurant', 'American Restaurant','Steakhouse','Restaurant',
                              'Indian Restaurant', 'Lounge', 'Japanese Restaurant', 'Seafood Restaurant',
                              'Pizza Place','Brewery',
                              'Ethiopian Restaurant', 'Breakfast Spot'],
        'Avg_Rating': [mr_avg, ar_avg, s_avg, r_avg, ir_avg, l_avg, jr_avg, sr_avg, pp_avg, b_avg, er_avg, bs_avg ]}

ratings_df = pd.DataFrame (data, columns = ['Venue Category','Avg_Rating'])

### 2e. Sort the venue categories by highest rating:

In [145]:
ratings_df.sort_values(by=['Avg_Rating'], ascending = False)

,Venue Category,Avg_Rating
6,Japanese Restaurant,8.600000
11,Breakfast Spot,8.000000
2,Steakhouse,7.400000
9,Brewery,7.400000
7,Seafood Restaurant,7.300000
8,Pizza Place,7.100000
10,Ethiopian Restaurant,7.050000
0,Mexican Restaurant,6.462500
1,American Restaurant,6.366667
3,Restaurant,6.100000


## 3. Map of all local restaurants in the dowtown Denver area:

In [158]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred around the Hotel

# add the restaurants as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map

## 4. Explore trending venues in the area (areas with the highest foot traffic):

In [159]:

url = 'https://api.foursquare.com/v2/venues/trending?client_id={}&client_secret={}&ll={},{}&v={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION)

# send GET request and get trending venues
results = requests.get(url).json()

In [160]:
if len(results['response']['venues']) == 0:
    trending_venues_df = 'No trending venues are available at the moment!'
    
else:
    trending_venues = results['response']['venues']
    trending_venues_df = json_normalize(trending_venues)

    # filter columns
    columns_filtered = ['name', 'categories'] + ['location.distance', 'location.city', 'location.postalCode', 'location.state', 'location.country', 'location.lat', 'location.lng']
    trending_venues_df = trending_venues_df.loc[:, columns_filtered]

    # filter the category for each row
    trending_venues_df['categories'] = trending_venues_df.apply(get_category_type, axis=1)

In [161]:
trending_venues_df

'No trending venues are available at the moment!'

In [162]:
if len(results['response']['venues']) == 0:
    venues_map = 'Cannot generate visual as no trending venues are available at the moment!'

else:
    venues_map = folium.Map(location=[latitude, longitude], zoom_start=15) # generate map centred around Ecco


    # add Ecco as a red circle mark
    folium.features.CircleMarker(
        [latitude, longitude],
        radius=10,
        popup='Ecco',
        fill=True,
        color='red',
        fill_color='red',
        fill_opacity=0.6
    ).add_to(venues_map)


    # add the trending venues as blue circle markers
    for lat, lng, label in zip(trending_venues_df['location.lat'], trending_venues_df['location.lng'], trending_venues_df['name']):
        folium.features.CircleMarker(
            [lat, lng],
            radius=5,
            poup=label,
            fill=True,
            color='blue',
            fill_color='blue',
            fill_opacity=0.6
        ).add_to(venues_map)
        
        
venues_map

'Cannot generate visual as no trending venues are available at the moment!'

## 5. Find the most common type of venue in downtown Denver:

In [163]:
def getNearbyVenues(names, latitudes, longitudes, radius=50):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [152]:
denver_venues = getNearbyVenues(names=dataframe_filtered['name'],
                                   latitudes=dataframe_filtered['lat'],
                                   longitudes=dataframe_filtered['lng']
                                  )

La Fiesta Mexican Restaurant
Rio Grande Mexican Restaurant
Randolph's Restaurant & Bar
The Corner Office Restaurant & Martini Bar
EDGE Restaurant And Bar At Four Seasons Hotel Denver
The Restaurant Source, a division of Bargreen Ellingson
Little India Restaurant - Champa Downtown
Pete's Satire Restaurant & Lounge
Altitude Restaurant
Denver Downtown Aquarium Restaurant
Mexico City Restaurant
Domo Japanese Country Foods Restaurant
Liberati Restaurant & Brewery
Lechuga's Italian Restaurant
Restaurant Connection Denver
Restaurant Depot
Benny's Restaurant and Tequila Bar
Solutions Lounge and Restaurant
Ethiopian Restaurant
D-Line Restaurant
New Peking Chinese Restaurant
Restaurant Burke
Abyssinia Ethiopian Restaurant
Las Delicias Authentic Mexican Restaurant
Mexico City Restaurant
Restaurant Rescue, Inc., of Colorado
Bella Vista Mexican Restaurant
El Charrito Mexican Restaurant
Camargos Tamales Restaurant
Foraged Restaurant
Far East Restaurant
Monnam Restaurant & Lounge
Choy Kee Chinese Res

In [153]:
denver_venues

,Venue,Venue Latitude,Venue Longitude,Venue Category
0,La Fiesta Mexican Restaurant,39.753410,-104.984636,Mexican Restaurant
1,Little Owl Coffee,39.750452,-104.999656,Coffee Shop
2,ChoLon Bistro,39.750562,-104.999673,Asian Restaurant
3,Freshcraft,39.750012,-104.999685,American Restaurant
4,D'Corazon,39.750116,-104.999667,Mexican Restaurant
...,...,...,...,...
106,Mellow Mushroom,39.748819,-104.996694,Pizza Place
107,Colorado Athletic Club - 16th Street,39.748789,-104.996578,Gym
108,Smashburger,39.748787,-104.996961,Burger Joint
109,Rocky Mountain Chocolate Factory,39.748621,-104.996913,Candy Store


In [164]:
denver_venues['Venue Category'].value_counts().head(15)

Mexican Restaurant      11
American Restaurant      9
Bar                      7
Marijuana Dispensary     5
Coffee Shop              4
Thai Restaurant          3
Sandwich Place           3
Pizza Place              3
Hotel                    3
Burger Joint             3
Brewery                  2
Spa                      2
Plaza                    2
Sushi Restaurant         2
Chinese Restaurant       2
Name: Venue Category, dtype: int64

## 6. Clustering the venues in dowtown Denver: 

In [165]:
denver_onehot = pd.get_dummies(denver_venues[['Venue Category']], prefix="", prefix_sep="")
denver_onehot['Venue'] = denver_venues['Venue'] 

In [166]:
# get a list of columns
cols = list(denver_onehot)
# move the column to head of list using index, pop and insert
cols.insert(0, cols.pop(cols.index('Venue')))

### 6a. Onehot encoding for cluster use:

In [167]:
denver_onehot = denver_onehot.reindex(columns= cols)

In [168]:
denver_grouped = denver_onehot.groupby('Venue').mean().reset_index()
denver_grouped

,Venue,American Restaurant,Aquarium,Arcade,Art Gallery,Asian Restaurant,BBQ Joint,Bar,Bookstore,Boutique,...,Sandwich Place,Seafood Restaurant,Shipping Store,Spa,Speakeasy,Steakhouse,Sushi Restaurant,Thai Restaurant,Theater,Vegetarian / Vegan Restaurant
0,7-Eleven,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,9Thai,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,Abarrotes La Herradura,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Abyssinia Ethiopian Restaurant,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Aloy Modern Thai,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99,Writer Square,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
100,denver recreational,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
101,il posto,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
102,ink! Coffee,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [169]:
num_top_venues = 5

for hood in denver_grouped['Venue']:
    temp = denver_grouped[denver_grouped['Venue'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})

In [170]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [172]:
num_top_venues = 1

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Venue']
for ind in np.arange(num_top_venues):
    try:
        columns.append('Venue Type'.format(ind+1, indicators[ind]))
    except:
        columns.append('Venue Type'.format(ind+1))

# create a new dataframe
denver_venues_sorted = pd.DataFrame(columns=columns)
denver_venues_sorted['Venue'] = denver_grouped['Venue']

for ind in np.arange(denver_grouped.shape[0]):
    denver_venues_sorted.iloc[ind, 1:] = return_most_common_venues(denver_grouped.iloc[ind, :], num_top_venues)

In [173]:
# set number of clusters
kclusters = 7

denver_grouped_clustering = denver_grouped.drop('Venue', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(denver_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] .astype(int) 

array([0, 3, 0, 0, 3, 0, 4, 0, 5, 5])

In [174]:
# add clustering labels
denver_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

denver_merged = denver_venues

denver_merged = denver_merged.join(denver_venues_sorted.set_index('Venue'), on='Venue', how = 'right')

denver_merged.head() # check the last columns

,Venue,Venue Latitude,Venue Longitude,Venue Category,Cluster Labels,Venue Type
0,La Fiesta Mexican Restaurant,39.753410,-104.984636,Mexican Restaurant,5,Mexican Restaurant
1,Little Owl Coffee,39.750452,-104.999656,Coffee Shop,1,Coffee Shop
2,ChoLon Bistro,39.750562,-104.999673,Asian Restaurant,0,Asian Restaurant
3,Freshcraft,39.750012,-104.999685,American Restaurant,4,American Restaurant
4,D'Corazon,39.750116,-104.999667,Mexican Restaurant,5,Mexican Restaurant


In [176]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=14)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]
integer = 1

#add the circle data point representing optimal location 
folium.features.CircleMarker(
    [39.7546069, -104.9783125],
    radius=10,
    color='white',
    popup='Predicted optimal restaurant location',
    fill = True,
    fill_color = 'black',
    fill_opacity = 0.6
).add_to(map_clusters)

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(denver_merged['Venue Latitude'], denver_merged['Venue Longitude'], denver_merged['Venue'], denver_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Cluster 0:

In [177]:
cluster_0 = denver_merged.loc[denver_merged['Cluster Labels'] == 0, denver_merged.columns[[0] + list(range(5, denver_merged.shape[1]))]]
cluster_0

,Venue,Venue Type
2,ChoLon Bistro,Asian Restaurant
6,Warwick Denver Hotel,Hotel
8,Ellie Caulkins Opera House,Opera House
9,DoubleTree by Hilton,Hotel
12,Four Seasons Spa,Spa
...,...,...
106,Mellow Mushroom,Pizza Place
107,Colorado Athletic Club - 16th Street,Gym
108,Smashburger,Burger Joint
109,Rocky Mountain Chocolate Factory,Candy Store


## Cluster 1:

In [179]:
denver_merged.loc[denver_merged['Cluster Labels'] == 1, denver_merged.columns[[0] + list(range(5, denver_merged.shape[1]))]]

,Venue,Venue Type
1,Little Owl Coffee,Coffee Shop
11,Starbucks,Coffee Shop
58,Fluid Coffee Bar,Coffee Shop
87,ink! Coffee,Coffee Shop


## Cluster 2:

In [180]:
denver_merged.loc[denver_merged['Cluster Labels'] == 2, denver_merged.columns[[0] + list(range(5, denver_merged.shape[1]))]]

,Venue,Venue Type
19,Cheba Hut Toasted Subs,Sandwich Place
54,Subway,Sandwich Place
98,Modern Market,Sandwich Place


## Cluster 3:

In [181]:
denver_merged.loc[denver_merged['Cluster Labels'] == 3, denver_merged.columns[[0] + list(range(5, denver_merged.shape[1]))]]

,Venue,Venue Type
55,9Thai,Thai Restaurant
56,Thai Monkey Club On Colfax,Thai Restaurant
60,Aloy Modern Thai,Thai Restaurant


## Cluster 4:

In [182]:
denver_merged.loc[denver_merged['Cluster Labels'] == 4, denver_merged.columns[[0] + list(range(5, denver_merged.shape[1]))]]

,Venue,Venue Type
3,Freshcraft,American Restaurant
7,Randolph's Restaurant & Bar,American Restaurant
10,The Corner Office Restaurant & Martini Bar,American Restaurant
13,Nickel,American Restaurant
38,The Cheesecake Factory,American Restaurant
57,D Bar Denver,American Restaurant
74,Vesta Dipping Grill,American Restaurant
82,"ViewHouse Eatery, Bar & Rooftop",American Restaurant
97,Appaloosa Grill,American Restaurant


## Cluster 5:

In [183]:
denver_merged.loc[denver_merged['Cluster Labels'] == 5, denver_merged.columns[[0] + list(range(5, denver_merged.shape[1]))]]

,Venue,Venue Type
0,La Fiesta Mexican Restaurant,Mexican Restaurant
4,D'Corazon,Mexican Restaurant
5,Rio Grande Mexican Restaurant,Mexican Restaurant
42,Benny's Restaurant and Tequila Bar,Mexican Restaurant
59,Las Delicias,Mexican Restaurant
64,Mexico City Restaurant,Mexican Restaurant
70,Mexico City Restaurant,Mexican Restaurant
67,Bella Vista Mexican Restaurant,Mexican Restaurant
73,Camargos Tamales Restaurant,Mexican Restaurant
88,Santiago's Mexican Restaurant,Mexican Restaurant


## Cluster 6:

In [184]:
denver_merged.loc[denver_merged['Cluster Labels'] == 6, denver_merged.columns[[0] + list(range(5, denver_merged.shape[1]))]]

,Venue,Venue Type
48,First Draft Taproom & Kitchen,Bar
50,The Matchbox,Bar
52,Lustre Pearl,Bar
63,Ignite Kitchen + Cocktails,Bar
69,Ignite Kitchen + Cocktails,Bar
78,Econo Mart Liquor,Bar
105,Satellite Bar,Bar
